In [1]:
import os
import random
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
from torch_geometric.transforms import NormalizeFeatures
import optuna
from optuna.trial import TrialState

import torchmetrics


from src.models import GAT, GraphSAGE, GIN
from src.utils import train_model, test_model, train_constrative_model, valid_model
from src.mean_average_distance import MAD, MADGap
from src.virtualnode import VirtualClassNode_init, UnidirectionalVirtualClassNode_init, VirtualClassNode, UnidirectionalVirtualClassNode

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [2]:
SAVE_PATH = 'results'
EARLY_STOPPING = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset = CitationFull(root='dataset/Cora', name='Cora', transform=NormalizeFeatures())
device

device(type='cuda', index=0)

In [3]:
data = dataset[0]
df = pd.DataFrame(data.x)
df['y'] = data.y
train, valid = train_test_split(df, stratify=df.y, test_size=0.4)
valid, test = train_test_split(valid, stratify=valid.y, test_size=0.5)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True
data.valid_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.valid_mask[valid.index]=True
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask[test.index]=True
data.to(device)

Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793], train_mask=[19793], valid_mask=[19793], test_mask=[19793])

In [4]:
mad = MAD(device=device, global_flag=True)
madgap = MADGap(device, 3, 8)

In [5]:
f1score = torchmetrics.F1Score(task = "multiclass", average="macro", num_classes=dataset.num_classes).to(device)

In [6]:
tuning_result = pd.DataFrame({
                            'trial' : [],
                            'model' : [],
                            'virtualnode' : [],
                            'vnode_init' : [],
                            'temperature' : [],
                            'constrative coef' : [],
                            'gaussian_noise_scale' : [],
                            'lr' : [],
                            'train_acc' : [],
                            'train_loss' : [],
                            'val_acc' : [],
                            'val_loss' : [],
                            'test_acc' : [],
                            'macro f1' : [],
                            'micro f1'
                            'minor f1' : [],
                            'mad' : [],
                            'madgap' : []                            
                            })

tuning_result.to_csv("training_res.csv", index=False)

In [7]:
def objective(trial):

    Vnodes = trial.suggest_categorical("vnode", ["uni", "bi"])

    # vnode_init = trial.suggest_categorical("class_mean", ["random", "zero"])
    
    if Vnodes == "uni":
        vc = UnidirectionalVirtualClassNode()
        data_for_tuning = vc.forward(data)
    else:
        vc = VirtualClassNode()
        data_for_tuning = vc.forward(data)
        
    temperature = trial.suggest_float("temperature", 0.1, 10)
    constrative_coef = trial.suggest_loguniform("contrastive_loss_coef", 1e-5, 1)
    cnode_weight = trial.suggest_loguniform("cnode_weight", 1, 10)
    lr = trial.suggest_loguniform("lr", 5e-4, 1e-2)
    positive_sampling = trial.suggest_categorical("positive_sampling", [True, False])

    g_noise = trial.suggest_categorical("g_noise", [0.0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])

    #model_ = trial.suggest_categorical("model", ["sage", "gat", "gin"])
    #if model_ == "sage":
    model = GraphSAGE(in_channels=dataset.num_features, hidden_channels=256, number_of_classes=dataset.num_classes, num_of_hidden_layers=4, device=device, noise_level=g_noise)
    #elif model_ == "gat":
    #    model = GAT(in_channels=dataset.num_features, hidden_channels=476, number_of_classes=dataset.num_classes, num_of_hidden_layers=4, device=device, heads=1, noise_level=g_noise)
    #elif model_ == "gin":
    #    model = GIN(in_channels=dataset.num_features, hidden_channels=415, number_of_classes=dataset.num_classes, num_of_hidden_layers=4, device=device, noise_level=g_noise)
    print('='*110)
    print(f'VC : {Vnodes}, lr : {lr:.5f}, temp : {temperature:.5f}, constrative coef : {constrative_coef:.5f}')
    print(f'noise : {g_noise:.4f}, cnode weight: {cnode_weight:.2f}, positive_sampling: {positive_sampling} ')
    max_loss = 10000
    early_stopping_count = 0
    print(f'Model: {model.name} | Number of parameters: {model.get_n_params()}')
    print('')
    model = model.to(device)
    data_for_tuning = data_for_tuning.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
    losses = []
    accs = []
    val_losses = []
    val_accs = []
    for epoch in range(2000):
        loss, acc = train_constrative_model(model, data_for_tuning, optimizer, criterion, 
                                            constrative_coef=constrative_coef, 
                                            temperature=temperature, 
                                            cnode_weight=cnode_weight,
                                            positive_sampling=positive_sampling)
        losses.append(loss.item())
        accs.append(100*acc)
        val_loss, val_acc = valid_model(model, data_for_tuning, criterion, 
                                        constrative_coef=constrative_coef, temperature=temperature, 
                                        cnode_weight=cnode_weight,
                                        positive_sampling = positive_sampling)
        val_accs.append(100*val_acc)
        if val_loss < max_loss:
            max_loss = val_loss
            early_stopping_count = 0
        else:
            early_stopping_count += 1
            if early_stopping_count > EARLY_STOPPING:
                print("Early stopping..")
                break
        if epoch%20==0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Train Acc: {100*acc:.2f}, Valid Loss: {val_loss:.4f}, Valid Acc: {100*val_acc:.2f}')
        if epoch > 1000:
            if val_acc < 0.4:
                print('underfitting...')
                break
    report = test_model(model, data_for_tuning)
    result = pd.DataFrame(report).T
    result_sliced = result.iloc[:-3 if len(result) < 23 else 20, :]
    test_acc = result.loc['accuracy'][0]
    result.loc['minorities-f1',:] = result_sliced.mean(axis=0)
    result.to_csv(os.path.join(SAVE_PATH, f'{model.name}_layers{model.num_of_hidden_layers}_neurons{model.hidden_channels}'+'.csv'))
    result = model(data_for_tuning.x.to(device), data_for_tuning.edge_index.to(device))[0].cpu()
    global_mad = mad(result).item()
    mad_gap = madgap(result, data_for_tuning.edge_index).item()

    model.eval()
    out, _ = model(data_for_tuning.x, data_for_tuning.edge_index)
    pred = out.argmax(dim=-1)
    f1 = f1score(data_for_tuning.y[data_for_tuning.valid_mask], pred[data_for_tuning.valid_mask])

    exp_result_dict = {
        'trial' : trial.number,
        'model' : model.name,
        'virtualnode' : vc,
        #'vnode_init' : vnode_init,
        'temperature' : temperature,
        'constrative coef' : constrative_coef,
        'gaussian_noise_scale'  : g_noise,
        'lr' : lr,
        'train_acc' : acc,
        'train_loss' : loss,
        'val_acc' : val_acc,
        'val_loss' : val_loss,
        'test_acc' : test_acc,
        'macro f1' : pd.DataFrame(report).T.loc['macro avg', 'f1-score'],
        'micro f1' : pd.DataFrame(report).T.loc['weighted avg', 'f1-score'],
        'minor f1' : pd.DataFrame(report).T[:-3].sort_values(by='support', ascending=False)[-11:].mean()['f1-score'],
        'mad' : global_mad,
        'madgap' : mad_gap                            
    }
    
    
    tuning_result = pd.read_csv("training_res.csv")
    exp_result_dict = pd.DataFrame(exp_result_dict, index=[trial.number])
    pd.concat([tuning_result, exp_result_dict], axis=0).to_csv("training_res.csv", index=False)
    
    torch.save(model.state_dict() , "model_file/{}_{}.pt".format(model.name, str(trial.number)))
                    
    print(f'global_mad: {global_mad}')
    print(f'madgap: {mad_gap}')
    print(f'Test Acc: {100*test_acc}')
    
    print('==========================================', end='\n\n')

    return f1


In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)


pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2023-06-06 18:58:24,891] A new study created in memory with name: no-name-bf4c0ee3-51d4-48b2-b61c-3dab6c25493f


VC : uni, lr : 0.00443, temp : 6.74325, constrative coef : 0.04142
noise : 0.0100, cnode weight: 2.67, positive_sampling: False 
Model: GraphSAGE | Number of parameters: 4853760

Epoch: 000, Train Loss: 11.6673, Train Acc: 1.07, Valid Loss: 11.7965, Valid Acc: 0.67
Epoch: 020, Train Loss: 10.1794, Train Acc: 8.30, Valid Loss: 9.5719, Valid Acc: 9.63
Epoch: 040, Train Loss: 6.0827, Train Acc: 21.62, Valid Loss: 4.2143, Valid Acc: 43.16
Epoch: 060, Train Loss: 4.6247, Train Acc: 31.50, Valid Loss: 2.8170, Valid Acc: 50.43
Epoch: 080, Train Loss: 3.9348, Train Acc: 35.16, Valid Loss: 2.3327, Valid Acc: 52.89
Epoch: 100, Train Loss: 3.4096, Train Acc: 38.13, Valid Loss: 2.2714, Valid Acc: 55.17
Epoch: 120, Train Loss: 3.0833, Train Acc: 42.02, Valid Loss: 1.9089, Valid Acc: 59.54
Epoch: 140, Train Loss: 2.6831, Train Acc: 43.68, Valid Loss: 1.8991, Valid Acc: 59.52
Epoch: 160, Train Loss: 2.5692, Train Acc: 45.68, Valid Loss: 1.7798, Valid Acc: 59.02
Epoch: 180, Train Loss: 2.3538, Train A

[I 2023-06-06 18:59:25,982] Trial 0 finished with value: 0.5634709000587463 and parameters: {'vnode': 'uni', 'temperature': 6.7432520917119065, 'contrastive_loss_coef': 0.041418693488941095, 'cnode_weight': 2.6731093659413068, 'lr': 0.004434228752407976, 'positive_sampling': False, 'g_noise': 0.01}. Best is trial 0 with value: 0.5634709000587463.


global_mad: 0.7767740488052368
madgap: 0.5315070748329163
Test Acc: 63.45036625410457

VC : uni, lr : 0.00119, temp : 9.61167, constrative coef : 0.01037
noise : 0.0010, cnode weight: 8.36, positive_sampling: False 
Model: GraphSAGE | Number of parameters: 4853760

Epoch: 000, Train Loss: 35.5532, Train Acc: 1.11, Valid Loss: 35.5698, Valid Acc: 0.79
Epoch: 020, Train Loss: 35.2005, Train Acc: 6.94, Valid Loss: 35.2060, Valid Acc: 10.23
Epoch: 040, Train Loss: 21.7880, Train Acc: 13.96, Valid Loss: 20.6279, Valid Acc: 16.41
Epoch: 060, Train Loss: 7.9307, Train Acc: 30.82, Valid Loss: 6.6412, Valid Acc: 34.40
Epoch: 080, Train Loss: 4.4493, Train Acc: 40.28, Valid Loss: 3.7983, Valid Acc: 46.07
Epoch: 100, Train Loss: 3.3913, Train Acc: 47.81, Valid Loss: 2.8006, Valid Acc: 52.84
Epoch: 120, Train Loss: 2.8758, Train Acc: 52.48, Valid Loss: 2.3444, Valid Acc: 56.17
Epoch: 140, Train Loss: 2.5216, Train Acc: 55.35, Valid Loss: 2.0611, Valid Acc: 58.05
Epoch: 160, Train Loss: 2.3391, Tra

[I 2023-06-06 19:03:32,697] Trial 1 finished with value: 0.6500105857849121 and parameters: {'vnode': 'uni', 'temperature': 9.611671887856675, 'contrastive_loss_coef': 0.01037447993783241, 'cnode_weight': 8.355735441468088, 'lr': 0.001192096299429533, 'positive_sampling': False, 'g_noise': 0.001}. Best is trial 1 with value: 0.6500105857849121.


...

global_mad: 0.8215625882148743
madgap: 0.48819810152053833
Test Acc: 69.86612781005304

VC : uni, lr : 0.00213, temp : 6.74101, constrative coef : 0.01321
noise : 0.0010, cnode weight: 7.66, positive_sampling: True 
Model: GraphSAGE | Number of parameters: 4853760

Epoch: 000, Train Loss: 32.6454, Train Acc: 1.11, Valid Loss: 32.6688, Valid Acc: 3.92
Epoch: 020, Train Loss: 28.7649, Train Acc: 10.41, Valid Loss: 27.4016, Valid Acc: 10.40
Epoch: 040, Train Loss: 7.8805, Train Acc: 31.02, Valid Loss: 6.4783, Valid Acc: 35.99
Epoch: 060, Train Loss: 4.0652, Train Acc: 45.48, Valid Loss: 3.2180, Valid Acc: 50.34
Epoch: 080, Train Loss: 2.7626, Train Acc: 52.74, Valid Loss: 2.2916, Valid Acc: 56.07
Epoch: 100, Train Loss: 2.2052, Train Acc: 59.00, Valid Loss: 1.7593, Valid Acc: 61.06
Epoch: 120, Train Loss: 1.9141, Train Acc: 62.19, Valid Loss: 1.5017, Valid Acc: 62.92
Epoch: 140, Train Loss: 1.7042, Train Acc: 63.98, Valid Loss: 1.3188, Valid Acc: 64.83
Epoch: 160, Train Loss: 1.5751, Trai

[I 2023-06-07 01:41:06,166] Trial 98 finished with value: 0.6592079401016235 and parameters: {'vnode': 'uni', 'temperature': 6.741007139874393, 'contrastive_loss_coef': 0.013214622024945062, 'cnode_weight': 7.6633848619966445, 'lr': 0.0021332694303884347, 'positive_sampling': True, 'g_noise': 0.001}. Best is trial 64 with value: 0.6664988994598389.


global_mad: 0.8620078563690186
madgap: 0.4940248131752014
Test Acc: 70.11871684768882

VC : bi, lr : 0.00415, temp : 6.59362, constrative coef : 0.00848
noise : 0.0010, cnode weight: 9.47, positive_sampling: True 
Model: GraphSAGE | Number of parameters: 4853760

Epoch: 000, Train Loss: 40.2903, Train Acc: 0.99, Valid Loss: 40.2943, Valid Acc: 6.88
Epoch: 020, Train Loss: 20.8862, Train Acc: 21.31, Valid Loss: 18.0608, Valid Acc: 20.77
Epoch: 040, Train Loss: 3.3427, Train Acc: 56.53, Valid Loss: 3.1334, Valid Acc: 47.60
Epoch: 060, Train Loss: 2.4882, Train Acc: 60.53, Valid Loss: 2.3258, Valid Acc: 50.43
Epoch: 080, Train Loss: 1.3977, Train Acc: 71.65, Valid Loss: 1.1531, Valid Acc: 58.65
Epoch: 100, Train Loss: 1.1740, Train Acc: 73.91, Valid Loss: 0.8579, Valid Acc: 62.03
Epoch: 120, Train Loss: 1.0040, Train Acc: 76.89, Valid Loss: 0.7590, Valid Acc: 62.92
Epoch: 140, Train Loss: 0.8658, Train Acc: 78.95, Valid Loss: 0.6562, Valid Acc: 64.53
Epoch: 160, Train Loss: 0.7893, Train 

[I 2023-06-07 01:46:13,905] Trial 99 finished with value: 0.6187418103218079 and parameters: {'vnode': 'bi', 'temperature': 6.593622201501682, 'contrastive_loss_coef': 0.008479609721183931, 'cnode_weight': 9.470094995222285, 'lr': 0.004150589014689215, 'positive_sampling': True, 'g_noise': 0.001}. Best is trial 64 with value: 0.6664988994598389.


global_mad: 0.781170666217804
madgap: 0.4934564232826233
Test Acc: 66.05203334175297



In [9]:


print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  0
  Number of complete trials:  100
Best trial:
  Value:  0.6664988994598389
  Params: 
    vnode: uni
    temperature: 6.569987463649486
    contrastive_loss_coef: 0.013821647940330835
    cnode_weight: 7.345715586378222
    lr: 0.0036288136927196597
    positive_sampling: True
    g_noise: 0.001
